In [1]:
from nltk.parse import CoreNLPParser
from nltk.tree import Tree
import re
from input_sentence import *

count=0

In [2]:
def traverse_tree(tree):
    '''
    Traverses a tree and outputs the tree head with numeric word order suffices attached
    '''
    global count
    leaf=tree.leaves()
    if len(leaf)==1:
        if tree.height()==2 and tree.label()!='.':
            count=count+1
            tree[0]=tree[0]+' - '+str(count)
    for subtree in tree:
        if type(subtree) == Tree:
            traverse_tree(subtree)
    return tree

def find_head(phrase):
    '''
    Give a phrase returns the head of the phrase
    Output format - head word (string)
    '''
    pos_tags=[]
    alltags=[]
    p=0
    leaf=1
    for i in phrase:
        if len(i)>1:
            leaf=0
    if leaf:
        for j in precedence_list:
            if p==1:
                break
            for word,pos in phrase.pos():        
                noun=re.match(j,pos)
                if noun and noun[0]:
                    return word
                    p=1
                    break
    else:
        for j in precedence_list:
            for subtree in phrase:
                pos=subtree.label().split()[0]
                noun=re.match(j,pos)
                if noun and noun[0]:
                    if len(subtree.label().split())>1:
                        return ' '.join(subtree.label().split()[1:])
                    else:
                        for word,pos in subtree.pos():
                            return word

def dep_tree(tree):
    '''
    Main function to call the subtrees of the tree for head marking in a recursive fashion
    Output - Tree with head words marked
    '''
    global temp
    for subtree in tree:
        if type(subtree) == Tree:
            dep_tree(subtree)
    if len(tree)==1 and tree.height()>2 and len(tree.leaves())==1:
        leaf=tree.leaves()
        templabel=tree.label()
        newlabel=templabel+ ' ' + leaf[0]
        tree.set_label(newlabel)
    if len(tree)!=len(temp):
        leaf=tree.leaves()
        if len(leaf)>1:
            head=find_head(tree)
            if not head:
                raise Exception('There are certain phrases for which I cannot find the heads as I do not have all the rules.  Please add rules and try again')

            templabel=tree.label()
            newlabel=templabel+' '+head
            tree.set_label(newlabel)
        elif len(leaf)==1:
            templabel=tree.label()
            newlabel=templabel+ ' ' + leaf[0]
            tree.set_label(newlabel)
    else:
        return tree
    
def dep_list(tree,deplist):
    '''
    Function to set up dependency links. The dependency links are appended in a list for simplicity
    '''
    global temp
    global treehead
    for subtree in tree:
        if type(subtree) == Tree:
            dep_list(subtree,deplist)
    children=tree.leaves()
    if tree.height()>2:
        if len(children)!=len(temp.leaves()):
            for j in children:
                if j!=' '.join(tree.label().split()[1:]) and (j not in [i[1] for i in deplist]):
                    deplist.append([' '.join(tree.label().split()[1:]),j])
    return deplist

def sorting(elem):
    '''
    helper function to sort the dependencies according to their word suffices
    '''
    return elem[1].split()[2]


In [3]:
parser = CoreNLPParser(url='http://localhost:9000')
#Read the sentence and form the parse tree
parser_output_splitted=parser.parse(sentence.split())

for j in parser_output_splitted:
    temp=j

### Part 1

In [4]:
parser_output=traverse_tree(temp)
print(parser_output)

(ROOT
  (S
    (NP (PRP$ My - 1) (NN dog - 2))
    (ADVP (RB also - 3))
    (VP (VBZ likes - 4) (NP (JJ eating - 5) (NN sausage - 6)))
    (. .)))


### Part 2

In [5]:
precedence_list=['VP','VB','VBG','VBZ',r'V*','NP','NN',r'N*','ADVP','JJ','P','RB','PRP$','PRP']
deptree=dep_tree(parser_output)
print(deptree)

(ROOT
  (S likes - 4
    (NP dog - 2 (PRP$ My - 1) (NN dog - 2))
    (ADVP also - 3 (RB also - 3))
    (VP likes - 4
      (VBZ likes - 4)
      (NP sausage - 6 (JJ eating - 5) (NN sausage - 6)))
    (. .)))


### Part 3

In [6]:
for subtree in deptree:
    treehead=subtree.label()
deplist=[]
deplist=dep_list(deptree,deplist)
deplist.append(['ROOT - 0',' '.join(treehead.split()[1:])])
print(deplist)

[['dog - 2', 'My - 1'], ['sausage - 6', 'eating - 5'], ['likes - 4', 'sausage - 6'], ['ROOT - 0', 'likes - 4']]


### Part 4

In [7]:
deplist=sorted(deplist,key= sorting)
print(deplist)

[['dog - 2', 'My - 1'], ['ROOT - 0', 'likes - 4'], ['sausage - 6', 'eating - 5'], ['likes - 4', 'sausage - 6']]
